In [40]:
import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pyspark

plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)
%matplotlib inline

In [41]:
train = pd.read_csv('data/trips_train_with_stationweather.csv', low_memory=False)
train_filtered = pd.read_csv('data/trips_train_with_stationweather_filtered.csv', low_memory=False)
test = pd.read_csv('data/trips_test_with_stationweather.csv', low_memory=False)

In [42]:
print train.shape
train.head(2)


(549961, 17)


,duration,season,start_station_id,end_station_id,start_hour,day_of_week,subscription_type,mean_temperature_f,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,events,wind_dir_degrees
0,396,summer,50,60,8,3,Subscriber,72.0,55.0,57.0,30.01,10.0,6.0,29.0,0,Clear,272.0
1,981,summer,50,82,9,3,Subscriber,72.0,55.0,57.0,30.01,10.0,6.0,29.0,0,Clear,272.0


In [43]:
test.head(2)

,trip_id,season,start_station_id,end_station_id,start_hour,day_of_week,subscription_type,mean_temperature_f,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,events,wind_dir_degrees
0,504737,autumn,60,71,11,5,Customer,65.0,54.0,64.0,29.94,10.0,6.0,28.0,0,Clear,285.0
1,505036,autumn,60,61,17,5,Subscriber,65.0,54.0,64.0,29.94,10.0,6.0,28.0,0,Clear,285.0


In [44]:
group_names = ['1', '2', '3', '4', '5']
train['mean_temperature_f'] = pd.qcut(train['mean_temperature_f'], 5, labels=group_names)
train[['mean_temperature_f', 'duration']].groupby(['mean_temperature_f'], as_index=False).mean().sort_values(by='mean_temperature_f', ascending=True)

,mean_temperature_f,duration
0,1,1009.471835
1,2,984.504272
2,3,1245.619184
3,4,1179.927536
4,5,1129.538014


In [45]:
group_names = ['1', '2', '3', '4']
train['mean_dew_point_f'] = pd.qcut(train['mean_dew_point_f'], 4, labels=group_names)
train[['mean_dew_point_f', 'duration']].groupby(['mean_dew_point_f'], as_index=False).mean().sort_values(by='mean_dew_point_f', ascending=True)

,mean_dew_point_f,duration
0,1,1042.688781
1,2,1051.163972
2,3,1076.855895
3,4,1318.130032


In [46]:
group_names = ['1', '2', '3', '4']
train['mean_humidity'] = pd.qcut(train['mean_humidity'], 4, labels=group_names)
train[['mean_humidity', 'duration']].groupby(['mean_humidity'], as_index=False).mean().sort_values(by='mean_humidity', ascending=True)

,mean_humidity,duration
0,1,1098.271584
1,2,1121.003302
2,3,1085.310515
3,4,1133.534873


In [47]:
group_names = ['1', '2', '3', '4']
train['mean_sea_level_pressure_inches'] = pd.qcut(train['mean_sea_level_pressure_inches'], 4, labels=group_names)
train[['mean_sea_level_pressure_inches', 'duration']].groupby(['mean_sea_level_pressure_inches'], as_index=False).mean().sort_values(by='mean_sea_level_pressure_inches', ascending=True)


,mean_sea_level_pressure_inches,duration
0,1,1087.670092
1,2,1158.508659
2,3,1061.014187
3,4,1146.956470


In [48]:
group_names = ['1', '2', '3', '4']
train['mean_wind_speed_mph'] = pd.qcut(train['mean_wind_speed_mph'], 4, labels=group_names)
train[['mean_wind_speed_mph', 'duration']].groupby(['mean_wind_speed_mph'], as_index=False).mean().sort_values(by='mean_wind_speed_mph', ascending=True)

,mean_wind_speed_mph,duration
0,1,1110.747702
1,2,1220.228930
2,3,998.600309
3,4,1097.121257


In [49]:
group_names = ['1', '2', '3', '4', '5', '6']
train['max_gust_speed_mph'] = pd.qcut(train['max_gust_speed_mph'], 6, labels=group_names)
train[['max_gust_speed_mph', 'duration']].groupby(['max_gust_speed_mph'], as_index=False).mean().sort_values(by='max_gust_speed_mph', ascending=True)

,max_gust_speed_mph,duration
0,1,1047.843041
1,2,1265.753621
2,3,1053.089657
3,4,1086.799882
4,5,1095.189937
5,6,1013.928496


In [50]:
group_names = ['1', '2', '3', '4', '5', '6']
train['wind_dir_degrees'] = pd.qcut(train['wind_dir_degrees'], 6, labels=group_names)
train[['wind_dir_degrees', 'duration']].groupby(['wind_dir_degrees'], as_index=False).mean().sort_values(by='wind_dir_degrees', ascending=True)

,wind_dir_degrees,duration
0,1,1219.980691
1,2,964.206437
2,3,1014.358952
3,4,1058.948644
4,5,1115.633949
5,6,1316.708177


In [51]:
def pct_rank_qcut(series, n):
    edges = pd.Series([float(i) / n for i in range(n + 1)])
    f = lambda x: (edges >= x).argmax()
    return series.rank(pct=1).apply(f)

train['mean_visibility_miles'] = train['mean_visibility_miles'].astype(float)
train['mean_visibility_miles'] = pct_rank_qcut(train.mean_visibility_miles, 5)

train[['mean_visibility_miles', 'duration']].groupby(['mean_visibility_miles'], as_index=False).mean().sort_values(by='mean_visibility_miles', ascending=True)

,mean_visibility_miles,duration
0,0,4314.302521
1,1,970.738084
2,4,1137.127259
3,5,4002.273876


In [52]:
    train_filtered['mean_temperature_f'] = pd.qcut(train_filtered['mean_temperature_f'], 5)
    train_filtered['mean_dew_point_f'] = pd.qcut(train_filtered['mean_dew_point_f'], 4)
    train_filtered['mean_humidity'] = pd.qcut(train_filtered['mean_humidity'], 4)
    train_filtered['mean_sea_level_pressure_inches'] = pd.qcut(train_filtered['mean_sea_level_pressure_inches'], 4)
    train_filtered['mean_wind_speed_mph'] = pd.qcut(train_filtered['mean_wind_speed_mph'], 4)
    train_filtered['max_gust_speed_mph'] = pd.qcut(train_filtered['max_gust_speed_mph'], 6)
    train_filtered['wind_dir_degrees'] = pd.qcut(train_filtered['wind_dir_degrees'], 6)
    '''CHEAQUEAR ESTO'''
    train_filtered['mean_visibility_miles'] = train_filtered['mean_visibility_miles'].astype(float)
    train_filtered['mean_visibility_miles'] = pct_rank_qcut(train_filtered.mean_visibility_miles, 5)

In [53]:
    test['mean_temperature_f'] = pd.qcut(test['mean_temperature_f'], 5)
    test['mean_dew_point_f'] = pd.qcut(test['mean_dew_point_f'], 4)
    test['mean_humidity'] = pd.qcut(test['mean_humidity'], 4)
    test['mean_sea_level_pressure_inches'] = pd.qcut(test['mean_sea_level_pressure_inches'], 4)
    test['mean_wind_speed_mph'] = pd.qcut(test['mean_wind_speed_mph'], 4)
    test['max_gust_speed_mph'] = pd.qcut(test['max_gust_speed_mph'], 6)
    test['wind_dir_degrees'] = pd.qcut(test['wind_dir_degrees'], 6)
    '''CHEAQUEAR ESTO'''
    test['mean_visibility_miles'] = test['mean_visibility_miles'].astype(float)
    test['mean_visibility_miles'] = pct_rank_qcut(test.mean_visibility_miles, 5)    

In [54]:
train.head(2)

,duration,season,start_station_id,end_station_id,start_hour,day_of_week,subscription_type,mean_temperature_f,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,events,wind_dir_degrees
0,396,summer,50,60,8,3,Subscriber,5,3,1,3,4,2,5,0,Clear,3
1,981,summer,50,82,9,3,Subscriber,5,3,1,3,4,2,5,0,Clear,3


In [55]:
features = ['season', 'subscription_type', 'mean_temperature_f', 'mean_dew_point_f',  'mean_humidity', 'mean_sea_level_pressure_inches','mean_visibility_miles', 'mean_wind_speed_mph', 'max_gust_speed_mph', 'precipitation_inches', 'events',  'wind_dir_degrees']
df_combined = pd.concat([train[features], test[features]])

In [58]:
    from sklearn import preprocessing    
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df_combined[feature])
        train[feature] = le.transform(train[feature])
        test[feature] = le.transform(test[feature])

ValueError: y contains new labels: ['autumn' 'spring' 'summer' 'winter']